# Échantillonnage direct de l'espace des motifs

### BRUNEAU Richard - VASLIN Pierre





In [6]:
import numpy as np
import scipy.special as sps
import math
import matplotlib
import pandas as pd
import random

class DataSet():
    def __init__(self,df:pd.DataFrame):
        self.df = df
        self.sizes = np.zeros(self.df.shape[0],dtype=int)
        for i in range(self.df.shape[0]):
            self.sizes[i] = self.df.iloc[i].count()
        # Sizes est pour connaitre la taille d'une ligne en o(1)
        # Pandas gère mal la variation du nombre de colonne dans une ligne dans un dataFrame, par concéquent 
        # il recalcule à chaque fois le nombre d'éléments non null o(n)


In [7]:
# Le dataframe pour les tests
df = pd.read_table("https://bitbucket.org/anesbendimerad/sigibbssamplingcode/raw/6699a50508fe177ee0c00dcc7d8e5390ee53688a/ItemsetDatasets/chess.txt", sep=" ",header=None)
del df[37]
ds = DataSet(df)
ds.df.head()

,0,1,2,3,4,5,6,7,8,9,...,27,28,29,30,31,32,33,34,35,36
0,1,3,5,7,9,11,13,15,17,19,...,56,58,60,62,64,66,68,70,72,74
1,1,3,5,7,9,12,13,15,17,19,...,56,58,60,62,64,66,68,70,72,74
2,1,3,5,7,9,12,13,16,17,19,...,56,58,60,62,64,66,68,70,72,74
3,1,3,5,7,9,11,13,15,17,20,...,56,58,60,62,64,66,68,70,72,74
4,1,3,5,7,9,11,13,15,17,19,...,56,58,60,62,64,66,68,70,72,74


## Question 1

In [8]:
def algoFrequences(ds:DataSet,nb_pattern)-> tuple:
  df = ds.df
  sizes = ds.sizes
  R = []
  IsInR = set()
  P: dict = dict()
  w = np.zeros(df.shape[0])
  totalW = 0
    
  # set les probas
  for i in range(len(w)):
    w[i] = math.pow(2,sizes[i])
    totalW += w[i]
  
  # on selectionne 
  while len(R) < nb_pattern:
    random_row = random.uniform(0,totalW)
    # On cherche la ligne
    row = 0
    v = 0
    for i in range(len(w)):
      if  v > random_row:
        row = i - 1
        break
      v += w[i]
    # On selectionne un motif 
    pattern = np.array(df.iloc[row][:sizes[row]])
    random_v = random.randint(1, len(pattern) - 1 )
    for i in range(len(pattern)- random_v):
      pattern = np.delete(pattern, random.randint(0, len(pattern) - 1 ))
    # On ajoute seulement les motifs non présent dans l'ensemble R
    IsInR.add(np.array2string(pattern))
    if len(IsInR) != len(R):
      R.append(pattern)
  return R

In [5]:
algoFrequences(ds,10)

[array([11, 15, 17, 19, 27, 34, 36, 40, 46, 66, 69, 70], dtype=int64),
 array([ 1,  3,  6,  7, 14, 24, 25, 27, 40, 43, 46, 50, 52, 56, 58, 60, 62,
        65, 66, 73, 74], dtype=int64),
 array([ 3,  9, 12, 21, 27, 31, 40, 56, 64], dtype=int64),
 array([19, 27, 48], dtype=int64),
 array([ 2,  3,  7,  9, 12, 14, 16, 17, 20, 21, 23, 25, 27, 29, 31, 34, 36,
        38, 40, 42, 44, 47, 48, 51, 52, 55, 56, 58, 60, 62, 64, 66, 69, 70,
        72, 75], dtype=int64),
 array([ 3,  5,  9, 11, 13, 15, 17, 19, 21, 24, 25, 27, 29, 31, 40, 43, 46,
        48, 51, 52, 54, 57, 58, 60, 62, 65, 66, 69, 70, 73, 74],
       dtype=int64),
 array([ 2,  5,  8,  9, 11, 14, 16, 17, 21, 25, 33, 34, 40, 51, 52, 54, 57,
        60, 62, 69, 71, 72], dtype=int64),
 array([ 3,  7, 15, 62, 70], dtype=int64),
 array([ 2,  5,  7,  9, 16, 20, 21, 23, 25, 34, 36, 38, 40, 42, 44, 47, 48,
        52, 55, 56, 58, 62, 64, 69, 75], dtype=int64),
 array([ 1,  3,  5, 14, 21, 34, 38, 40, 48, 64], dtype=int64)]

## Question 2

In [9]:
def algoArea(ds:DataSet,nb_pattern)-> tuple:
  df = ds.df
  sizes = ds.sizes
  R = []
  IsInR = set()
  
  # set les probas
  w = np.zeros(df[0].count(),dtype=np.ulonglong)
  totalW:np.ulonglong= 0
  for i in range(1, len(w)):
    w[i] = np.multiply(sizes[i], np.power(2, (sizes[i] - 1)))
    totalW += w[i]
  
  # on selectionne 
  while len(R) < nb_pattern:
    # On cherche la ligne
    random_row = random.uniform(0,totalW)
    v, row = 0,0
    for i in range(len(w)):
      if v > random_row:
        row = i - 1
        break
      v += w[i]
      
    # On set les probabilités de k (taille du motif)
    # On souhaite que le sous-ensemble est une taille calculée 
    # proportionnellement avec les tailles des datarecords (lignes) 
    ks = np.zeros(int(sizes[row]))
    totalK = 0
    for i in range(len(ks)):
        ks[i] = sps.binom(len(ks), i + 1)
        totalK += ks[i]
    #totalK = (len(ks) * (len(ks)+1))/2
    random_kp = random.uniform(0, totalK)
    # On cherche la ligne
    k, v = 0,0
    for i in range(len(ks)):
      if v > random_kp:
        k = i - 1
        break
      v += ks[i]
    
    pattern = np.array(df.iloc[row][:sizes[row]])
    for i in range(len(pattern)- k):
      pattern = np.delete(pattern, random.randint(0, len(pattern) - 1 ))
    
    # On ajoute seulement les motifs non présents dans l'ensemble R
    IsInR.add(np.array2string(pattern))
    if len(IsInR) != len(R):
      R.append(pattern)
  return R

In [7]:
algoArea(ds,10)

[array([ 5,  9, 13, 17, 19, 21, 23, 42, 44, 50, 54, 60, 64, 70],
       dtype=int64),
 array([ 5,  9, 11, 15, 17, 27, 31, 34, 40, 42, 44, 50, 52, 54, 62, 64],
       dtype=int64),
 array([ 1,  3,  7,  9, 19, 23, 29, 31, 40, 42, 44, 46, 52, 56, 58],
       dtype=int64),
 array([ 1,  3,  7,  9, 13, 25, 29, 31, 34, 36, 38, 42, 48, 54, 58, 60, 62,
        64, 68, 72, 74], dtype=int64),
 array([ 1,  7,  9, 17, 19, 21, 23, 25, 31, 34, 36, 38, 42, 52, 56, 60, 62,
        64, 68, 72], dtype=int64),
 array([ 5,  9, 15, 17, 21, 23, 27, 40, 42, 46, 50, 54, 60, 62, 66, 68, 70,
        74], dtype=int64),
 array([ 1,  5,  7,  9, 11, 23, 29, 48, 52, 58, 62, 70, 74], dtype=int64),
 array([ 9, 15, 27, 29, 31, 34, 44, 46, 50, 52, 62, 66, 68, 70, 72, 74],
       dtype=int64),
 array([ 3,  5,  7, 15, 23, 27, 29, 31, 34, 36, 40, 42, 48, 52, 54, 56, 58,
        60, 62, 64, 70], dtype=int64),
 array([ 1,  5,  7, 15, 17, 19, 25, 27, 31, 40, 42, 44, 56, 60, 68, 72],
       dtype=int64)]

## Question 3

In [16]:
def frequences(ds, patterns):
  df = ds.df
  sizes = ds.sizes
  frequencesP = np.zeros(len(patterns),dtype=float)
  indexs = np.zeros(len(patterns),dtype=int)
  lenPat = len(patterns)
  for i in range(df.shape[0]):
    indexs[True] = 0
    for j in range(ds.sizes[i]):
      for ip in range(lenPat):
        if (len(patterns[ip])!= indexs[ip] and 
            df.iloc[i][j] == patterns[ip][indexs[ip]]):
          indexs[ip] += 1
          if len(patterns[ip]) == indexs[ip]:
            frequencesP[ip] += 1.
  for i in range(len(frequencesP)):
    frequencesP[i] = (frequencesP[i])/float(df.shape[0])
  return frequencesP

In [9]:
patterns = algoFrequences(ds,10)

In [10]:
frequences(ds,patterns)

array([0.00125156, 0.00093867, 0.00031289, 0.00062578, 0.00031289,
       0.00125156, 0.00062578, 0.00031289, 0.00031289, 0.02033792])

In [17]:
def aire(ds, patterns):
  df = ds.df
  aireP = np.zeros(len(patterns),dtype=int)
  indexs = np.zeros(len(patterns),dtype=int)
  lenPat = len(patterns)
  for i in range(df.shape[0]):
    indexs[True] = 0
    for j in range(ds.sizes[i]):
      for ip in range(lenPat):
        if (len(patterns[ip])!= indexs[ip] and 
            df.iloc[i][j] == patterns[ip][indexs[ip]]):
          indexs[ip] += 1
          if len(patterns[ip]) == indexs[ip]:
            aireP[ip] += 1
  for i in range(len(aireP)):
    aireP[i] = aireP[i]*len(patterns[i])
  return aireP

In [12]:
patterns = algoArea(ds,10)

In [13]:
aire(ds,patterns)

array([ 529,  616,  897, 1424, 1482, 2227,  589, 2784, 2210,  304])

## Question 4 

Nous allons maintenant tester nos algorithmes avec des jeux de données suggérés dans le sujet.

In [14]:
dataset_1 = pd.read_table("https://bitbucket.org/anesbendimerad/sigibbssamplingcode/raw/6699a50508fe177ee0c00dcc7d8e5390ee53688a/ItemsetDatasets/chess.txt", sep=" ",header=None)
del dataset_1[37]
ds1 = DataSet(dataset_1)

# On réalise algoFrequence sur le dataset_1 5 fois.
dsFrequence1 = algoFrequences(ds1, 5)
# On ressort les fréquences 
freq1 = frequences(ds1, dsFrequence1)

# On réalise algoAire sur le dataset_1.
dsArea1 = algoArea(ds1, 5)
# On ressort les valeurs
area1 = aire(ds1, dsArea1)

In [18]:
dataset_2 = pd.read_fwf("https://www.philippe-fournier-viger.com/spmf/datasets/LEVIATHAN.txt", sep=" ",header=None,)
dataset_2 = dataset_2[0].str.split(' ', expand=True)
ds2 = DataSet(dataset_2)

# On réalise algoFrequence sur le dataset_2 5 fois.
dsFrequence2 = algoFrequences(ds2, 5)
# On ressort les fréquences 
freq2 = frequences(ds2, dsFrequence2)

# On réalise algoAire sur le dataset_2.
dsArea2 = algoArea(ds2, 5)
# On ressort les valeurs
area2 = aire(ds2, dsArea2)

In [20]:
dataset_3 = pd.read_fwf("https://www.philippe-fournier-viger.com/spmf/datasets/FIFA.txt", sep=" ",header=None,)
dataset_3 = dataset_3[0].str.split(' ', expand=True)
ds3 = DataSet(dataset_3)

# On réalise algoFrequence sur le dataset_3 5 fois.
dsFrequence3 = algoFrequences(ds3, 5)
# On ressort les fréquences 
freq3 = frequences(ds3, dsFrequence3)

# On réalise algoAire sur le dataset_3.
dsArea3 = algoArea(ds3, 5)
# On ressort les valeurs
area3 = aire(ds3, dsArea3)

In [25]:
dataset_4 = pd.read_fwf("https://www.philippe-fournier-viger.com/spmf/datasets/BIBLE.txt", sep=" ",header=None,)
dataset_4 = dataset_4[0].str.split(' ', expand=True)
ds4 = DataSet(dataset_4)

# On réalise algoFrequence sur le dataset_4 5 fois.
dsFrequence4 = algoFrequences(ds4, 5)
# On ressort les fréquences 
freq4 = frequences(ds4, dsFrequence4)

# On réalise algoAire sur le dataset_4.
dsArea4 = algoArea(ds4, 5)
# On ressort les valeurs
area4 = aire(ds4, dsArea4)

## Question 5

### Etude emprique

Nous allons maintenant étudier les résultats des datasets testé ci-dessus.

#### Dataset n°1

##### Algorithme de fréquence

Dans un premier temps, nous allons analyser l'algorithme de fréquence.

In [18]:
print("Nous avons extrait les motifs suivants:")
for dsF in dsFrequence1:
    print(dsF)
for f in freq1:
    print("La fréquence est de ", round(f*100,2), "%.")

Nous avons extrait les motifs suivants:
[ 3  5  7  9 12 14 15 18 20 22 23 25 28 29 31 34 36 38 40 42 44 46 48 50
 52 55 56 58 60 62 64 66 68 70 72 74]
[ 3 11 14 15 23 25 29 38 42 46 52 55 60 70 72]
[ 1  3  7  9 11 13 15 17 20 23 25 29 31 36 40 42 45 52 55 58 60 63 66 69
 72 75]
[ 7  9 14 21 25 36 39 48 51 54 56 58 60 62 66 74]
[ 2  7 12 13 18 20 22 24 27 29 34 38 40 44 47 50 52 54 56 58 60 62 64 68
 71 72 74]
La fréquence est de  0.03 %.
La fréquence est de  2.22 %.
La fréquence est de  0.09 %.
La fréquence est de  2.44 %.
La fréquence est de  0.13 %.


Nous constatons que nous avons une bonne variété des motifs. Les échantillons tirés sont également différents les uns des autres, nous n'avons pas de redondance. 
Le bémol que nous pouvons soulevé, c'est qu'il semblerait que la majorité des lignes commencent par un 1, hors dans notre échantillon, il n'y en a qu'une sur les cinq. La proportion du nombre de ligne terminant par 74 semblent respectées. 

Nous avons une fréquence de nos motifs qui oscille entre 0,03% et 2,44% ce qui donne une fréquence moyenne de 0,98%. 

##### Algorithme de l'aire

Dans un second temps, nous allons analyser l'algorithme de l'aire.

In [19]:
print("Nous avons extrait les motifs suivants:")
for dsA in dsArea1:
    print(dsA)
surface = dataset_1.shape
s = surface[0]*surface[1]
print("\nL'aire totale du dataset est de :", s, "\n")
for ar in area1: 
    print("L'aire des motifs précédants est de:", ar, " soit ", round((ar/s)*100, 2), "% du dataset.")

Nous avons extrait les motifs suivants:
[ 3  5  9 11 13 15 19 23 27 31 34 48 52 56 70 72]
[ 3 11 13 17 19 21 23 25 27 36 40 46 48 52 60 62 70 72]
[ 1  3  5  7 11 15 21 27 34 36 38 40 44 46 54 60 64 70 72 74]
[ 1  5 19 27 29 31 34 36 42 46 50 58 72]
[ 3  7  9 13 21 23 25 29 31 34 38 42 44 52 58 62 66 74]

L'aire totale du dataset est de : 118252 

L'aire des motifs précédants est de: 1600  soit  1.35 % du dataset.
L'aire des motifs précédants est de: 756  soit  0.64 % du dataset.
L'aire des motifs précédants est de: 180  soit  0.15 % du dataset.
L'aire des motifs précédants est de: 3354  soit  2.84 % du dataset.
L'aire des motifs précédants est de: 2052  soit  1.74 % du dataset.


Nous remarquons ici que l'aire des motifs varient énormément. Entre l'aire du motif le plus petit et l'aire du motif le plus grand il y a un rapport supérieur à 8. A nouveau, la méthode d'échantillonnage nous a permit d'obtenir une bonne diversité de motifs tirés. Une fois de plus, il semblerait que la part de ligne commençant par un 1 devrait être plus importante.

Il est important de préciser, et cette précision sera valable pour l'ensemble des analyses de l'aire, que l'aire du dataset est calculée en fonction de la ligne la plus grande. C'est une aire théorique et dans le cas d'un dataset de 10 lignes si une ligne à 10 valeurs et les autres une seule, le dataset aura une aire estimé de 10x10 = 100 et non de 9x1+10x1 = 19. 
Cette pour cela que nos valeurs en pourcentage du dataset sont extrêmement faibles, nous sommes trop sensible à la présence d'une ligne beaucoup plus grande que les autres.
Nous avons calculé l'aire ainsi afin de gagner du temps de calculs sur nos machines personnelles. 

#### Dataset n°2

##### Algorithme de fréquence 

Comme pour le dataset précédant, nous allons dans un premier temps analyser l'algorithme de fréquence. 

In [26]:
print("Nous avons extrait les motifs suivants:")
for dsF in dsFrequence2:
    print(dsF)
for f in freq2:
    print("La fréquence est de ", round(f*100,4), "%.")

Nous avons extrait les motifs suivants:
['1' '-1' '1' '1' '-1' '-1' '-1' '1' '-1' '1' '-1' '-1' '1' '-1' '-1' '-1'
 '-1' '1' '-1' '1' '-1' '1' '1' '1' '-1' '-1' '1' '-1' '-1' '-1' '1' '1'
 '-1' '-1' '1' '-1' '1' '1' '-1' '1' '1' '-1' '-1' '1' '-1' '-1' '1' '-1'
 '1' '-1' '2372' '-1' '-1' '-1' '1' '2375' '54' '2376' '17' '-1']
['1' '-1' '-1' '1' '-1' '-1' '1' '-1' '-1' '-1' '-1' '1' '-1' '1' '-1' '1'
 '1' '-1' '549' '-1' '1' '1' '-1' '1' '1' '-1' '1' '1' '-1' '-1' '1' '1'
 '1' '1' '1' '-1' '1' '363' '550' '1' '-1' '-1' '-1' '468' '-1' '555' '-1'
 '-1' '147' '-1' '-1' '500' '40' '29' '212' '559' '-2']
['1' '-1' '-1' '-1' '1' '-1' '-1' '-1' '1' '-1' '-1' '-1' '-1' '1' '-1'
 '1' '1' '1' '-1' '1' '-1' '-1' '1' '1' '5772' '-1' '-1' '1' '1' '-1' '-1'
 '1' '-1' '1' '1' '1' '-1' '-1' '1' '1' '1' '-1' '-1' '-1' '-1' '1' '-1'
 '-1' '-1' '-1' '-1' '2873' '3991' '1' '1' '-1' '961' '-1' '175' '8' '-1'
 '29' '-1' '-1' '124' '632' '-1' '-1' '124' '-1' '130' '-1' '26']
['1' '-1' '1' '-1' '1' '-1' '1' '

Bien que la fréquence soit faible, nous constatons ici qu'elle est identique pour tout les échantillons obtenus. 
Après une analyse visuelle du fichier brut, la part de "-1" semble supérieur sur nos échantillons que dans le fichier brut. 

Nous pouvons également noter que la présence de nombres négatifs n'influe pas sur le déroulement de notre algorithme. 

##### Algorithme de l'aire

Dans un second temps, nous allons analyser l'algorithme de l'aire.

In [22]:
print("Nous avons extrait les motifs suivants:")
for dsA in dsArea2:
    print(dsA)
surface = dataset_2.shape
s = surface[0]*surface[1]
print("\nL'aire totale du dataset est de :", s, "\n")
for ar in area2: 
    print("L'aire des motifs précédants est de:", ar, " soit ", round((ar/s)*100, 4), "% du dataset.")

Nous avons extrait les motifs suivants:
['781' '-1' '-1' '381' '-1' '1244' '-1' '8' '3224' '-1' '17' '-1' '52'
 '-1' '18' '-1' '8' '493' '103' '-1' '-1' '608' '-1' '14' '-1' '8' '-1'
 '-1' '-1' '-1' '97' '-1' '211' '-1' '8' '-1' '-2']
['36' '-1' '21' '-1' '-1' '5824' '265' '-1' '-1' '71' '307' '8376' '779'
 '-1' '-1' '-1' '-1' '-1' '21' '-1' '4105' '-1' '8' '-1' '3711' '-1' '381'
 '351' '-1' '1356']
['1' '-1' '-1' '-1' '1111' '-1' '103' '-1' '8' '-1' '1783' '-1' '-1' '122'
 '-1' '-1' '75' '-1' '-1' '2992' '-1' '29' '-1' '52' '-1' '-1' '1045' '-2']
['1' '-1' '1' '-1' '-1' '18' '-1' '-1' '3256' '-1' '71' '-1' '312' '20'
 '-1' '1338' '318' '-1' '104' '-1' '-1' '-1' '18' '209' '-1' '206' '-1'
 '51' '-1' '-1' '1979' '-1' '8' '-1' '3065' '-1' '-2']
['-1' '162' '-1' '7025' '54' '-1' '8' '2686' '-1' '-1' '6887' '-1' '2687'
 '-1' '21' '-1' '-1' '7026' '-1' '-1' '-1' '2138' '-1' '3' '-1' '-1' '177'
 '-1' '568' '5061']

L'aire totale du dataset est de : 1085124 

L'aire des motifs précédants est 

Encore une fois les valeurs présentes ici sont proches les une des autres. En revanche, pour ce dataset contrairement à l'algorithme de fréquence, nous avons une meilleure diversité des motifs tirés. 

On constate à nouveau que la présence de nombres négatifs n'a pas posé de problème.

#### Dataset n°3

##### Algorithme de fréquence 

Comme pour les datasets précédants, nous allons dans un premier temps analyser l'algorithme de fréquence.

In [23]:
print("Nous avons extrait les motifs suivants:")
for dsF in dsFrequence3:
    print(dsF)
for f in freq3:
    print("La fréquence est de ", round(f*100,4), "%.")

Nous avons extrait les motifs suivants:
['17' '-1' '-1' '17' '-1' '17' '-1' '-1' '17' '17' '-1' '17' '17' '-1'
 '17' '-1' '-1' '17' '-1' '17' '17' '-1' '-1' '-1' '17' '-1' '17' '-1'
 '-1' '17' '-1' '17' '-1' '-1' '17' '-1' '17' '-1' '17' '-1' '17' '-1'
 '17' '-1' '17' '-1' '17' '17' '17' '-1' '-1' '-1' '-1' '17' '-1' '17'
 '-1' '17' '17' '-1' '-1' '17' '17' '-1' '-1' '17' '17' '-1' '-1' '17'
 '-1' '17' '-1' '17' '-1' '-1' '-1' '17' '-1' '17' '17' '-1' '17' '17'
 '-1' '17' '-1' '17' '17' '-1' '17' '17' '-1' '17' '-1' '17' '-1' '17'
 '-1' '17' '-1' '17' '17' '-1' '17' '-1' '-1' '17' '-1' '-1' '-1' '17']
['17' '-1' '48' '-1' '33' '-1' '-1' '47' '-1' '45' '15' '-1' '32' '-1'
 '-1' '135' '147' '-1' '131' '-1' '-1' '13' '-1' '90' '8' '18' '30' '31'
 '-1' '67' '-1' '70' '68' '-1' '96' '-1' '50' '-1' '44' '-1' '-1' '14'
 '-1' '10' '-1' '21' '-1' '24' '-1' '59' '-1' '-1' '37' '36' '-1' '118'
 '-1' '97' '-1' '98' '-1' '756' '-1' '-1' '163' '-1' '25' '173' '-1' '304'
 '-1' '11' '-1' '55' '40' '-1

Comme pour le dataset n°2, nous obtenons une valeur pour la fréquence identique à tout les échantillons obtenus. Cela montre la régularité de l'algorithme de fréquence. 
La diversification des motifs semblent être plus importante qu'avec le dataset précédant.

##### Algorithme de l'aire
Dans un second temps, nous allons analyser l'algorithme de l'aire.

In [24]:
print("Nous avons extrait les motifs suivants:")
for dsA in dsArea3:
    print(dsA)
surface = dataset_3.shape
s = surface[0]*surface[1]
print("\nL'aire totale du dataset est de :", s, "\n")
for ar in area3: 
    print("L'aire des motifs précédants est de:", ar, " soit ", round((ar/s)*100, 4), "% du dataset.")

Nous avons extrait les motifs suivants:
['369' '2630' '-1' '2631' '80' '-1' '-1' '2547' '-1' '-1' '-1' '2634' '-1'
 '-1' '-1' '1005' '966' '-1' '-1' '-1' '-1' '939' '1129' '1020' '-1' '-1'
 '1180' '1150' '-1' '-1' '-2']
['222' '-1' '-1' '-1' '-1' '-1' '185' '229' '-1' '-1' '27' '-1' '-1' '51'
 '72' '121' '-1' '-1' '166' '253' '99' '-1' '193' '-1' '-1' '225' '-1'
 '-2']
['-1' '-1' '-1' '30' '-1' '97' '70' '-1' '147' '-1' '-1' '-1' '59' '181'
 '135' '90' '-1' '67' '-1' '117' '-1' '118' '-1' '13' '44' '82' '2' '-1'
 '98' '-1']
['142' '-1' '328' '-1' '264' '-1' '279' '-1' '43' '-1' '-1' '236' '165'
 '43' '202' '-1' '142' '236' '-1' '279' '-1' '-1' '264' '-1' '-1' '116'
 '-1' '277' '292' '376' '-1' '322' '749' '-1']
['15' '-1' '315' '-1' '33' '-1' '49' '-1' '30' '155' '70' '-1' '-1' '-1'
 '324' '98' '-1' '-1' '1' '-1' '-1' '21' '-1' '-1' '412' '-1' '57' '2'
 '-1' '37']

L'aire totale du dataset est de : 3578750 

L'aire des motifs précédants est de: 31  soit  0.0009 % du dataset.
L'aire des

Nous pouvons constater à nouveau que les valeurs sont très proches les une des autres. Les échantillons ont toujours des motifs diversifiés. Les motifs sont également tirés proportionnellement. 

#### Dataset n°4

##### Algorithme de fréquence 

Comme pour les datasets précédants, nous allons dans un premier temps analyser l'algorithme de fréquence.

In [27]:
print("Nous avons extrait les motifs suivants:")
for dsF in dsFrequence4:
    print(dsF)
for f in freq4:
    print("La fréquence est de ", round(f*100,4), "%.")

Nous avons extrait les motifs suivants:
['-1' '77' '-1' '38' '-1' '-1' '5579' '-1' '31' '-1' '38' '-1' '10' '-1'
 '-1' '46' '-1' '-1' '3997' '-1' '4' '-1' '669' '-1' '544' '-1' '2017'
 '-1' '38' '-1' '22' '661' '-1' '51' '456' '-1' '15' '-1' '61' '-1' '269'
 '-1' '10352' '-1' '38' '-1' '10' '-1' '46' '-1' '374' '356' '-1' '-1'
 '22' '-1' '-1' '-1' '46' '10' '-1' '1595' '-1' '31' '-1' '10' '-1' '2742'
 '-1' '46' '-1' '10' '-1' '318' '31' '-1' '10' '-1' '131' '-1' '-1' '10'
 '-1' '2531' '-1' '31' '-1' '218' '2513' '-1' '3' '-1' '-1' '188' '-1'
 '10' '-1' '926' '-1' '31' '-1' '218' '-1' '10' '-1' '222' '-1' '-1' '127'
 '-1' '-1' '10' '-1' '157' '10' '-1' '926' '-1' '456' '7165' '-1' '-1'
 '1585' '-1' '31' '-1' '-1' '2784' '-1' '456' '-1' '-1' '4618' '-1' '-1'
 '31' '-1']
['356' '-1' '-1' '14' '41' '46' '-1' '-1' '-1' '14' '-1' '15' '-1' '16'
 '1209' '-1' '8' '-1' '10' '-1' '955' '10' '5714' '-1' '1283' '-1' '15'
 '-1' '65' '8' '-1' '218' '-1' '-1' '2494' '-1' '8' '-1' '22' '-1' '325'
 '-1

Une fois de plus nous constatons que l'algorithme de fréquence ressort des valeurs identiques. Les motifs sont diversifiés. 

##### Algorithme de l'aire
Dans un second temps, nous allons analyser l'algorithme de l'aire.

In [28]:
print("Nous avons extrait les motifs suivants:")
for dsA in dsArea4:
    print(dsA)
surface = dataset_4.shape
s = surface[0]*surface[1]
print("\nL'aire totale du dataset est de :", s, "\n")
for ar in area4: 
    print("L'aire des motifs précédants est de:", ar, " soit ", round((ar/s)*100, 4), "% du dataset.")

Nous avons extrait les motifs suivants:
['-1' '-1' '14' '2123' '-1' '663' '-1' '305' '-1' '881' '-1' '38' '1134'
 '-1' '73' '-1' '12644' '3038' '-1' '2123' '31' '-1' '-1' '-1' '43' '384'
 '-1' '31' '-1' '121' '11354' '-2']
['-1' '-1' '-1' '2836' '-1' '-1' '560' '59' '12' '-1' '31' '456' '-1'
 '396' '-1' '2546' '55' '560' '-1' '5333' '-1' '-1' '55' '-1' '560' '5352']
['5736' '61' '-1' '8' '-1' '-1' '116' '-1' '12' '4228' '-1' '-1' '794'
 '-1' '2430' '-1' '560' '10' '-1' '4530' '-1' '46' '-1' '560' '-1' '8036'
 '-1' '-2']
['-1' '10' '-1' '-1' '-1' '-1' '535' '22' '11330' '583' '-1' '1141' '-1'
 '10' '2794' '-1' '46' '-1' '243' '-1' '-1' '10' '-1' '363' '-1' '46' '-1']
['356' '-1' '129' '-1' '10' '-1' '3736' '-1' '456' '15' '31' '-1' '10'
 '-1' '4139' '22' '-1' '4238' '116' '-1' '51' '-1' '61' '-1' '3318' '890'
 '-1' '1232' '31' '-1' '-2']

L'aire totale du dataset est de : 6364575 

L'aire des motifs précédants est de: 32  soit  0.0005 % du dataset.
L'aire des motifs précédants est de: 2

Nous constatons à nouveau pour ce dernier dataset que les valeurs retournées sont très proches les unes des autres. Les motifs sont tirés de manière diversifiée dans le dataset.

## Question 6

On a constaté que lorsque l'on avait une ligne beaucoup plus grande que les autres, les algorithmes de fréquence et d'aire favorisent la ligne la plus grande pour créer des motifs. Ce comportement est lié au poids attribué à la ligne la plus grande. Nous souhaitons trouver des motifs fréquents, donc nous souhaitons éviter des motifs qui s'appliquent seulement dans des grandes lignes. Nous avons trouver trois solutions:

*   Supprimer les grandes lignes du dataset
*   Réduire la taille des grandes lignes
*   Modifier le poids accorder à la ligne la plus grande, cette option n'implique pas de perte de données dans le dataset



In [29]:
dsetGL = pd.read_fwf("https://www.philippe-fournier-viger.com/spmf/datasets/LEVIATHAN.txt", sep=" ",header=None,)
df_gl= dsetGL[0].str.split(' ', expand=True)
ds_gl = DataSet(df_gl)
ds_gl.df.head()

,0,1,2,3,4,5,6,7,8,9,...,165,166,167,168,169,170,171,172,173,174
0,3,-1,28,-1,58,-1,61,-1,64,-1,...,None,None,None,None,None,None,None,None,None,None
1,4,-1,7,-1,23,-1,255,-1,3,-1,...,None,None,None,None,None,None,None,None,None,None
2,8,-1,10,-1,13,-1,14,-1,21,-1,...,None,None,None,None,None,None,None,None,None,None
3,11,-1,25,-1,40,-1,54,-1,55,-1,...,None,None,None,None,None,None,None,None,None,None
4,12,-1,39,-1,63,-1,72,-1,3,-1,...,None,None,None,None,None,None,None,None,None,None


## Cas où le dataset est chargé dans un df
Dans cette option, nous allons calculer la moyenne du nombre d'items par ligne et en fonction de cela, nous supprimerons les colonnes avec une taille supérieure à la taille moyenne du nombre d'items par ligne. 
Nous nous concentrons sur une partie du dataset, dans l'exemple 40%, afin de gagner du temps lors de l'exécution. Ce seuil est paramétrable dans l'appel à la fonction. 

In [27]:
ds_gl.df.shape

(20450, 175)

In [28]:
def resizeDataSet(ds,percentage):
  df = ds.df
  shape = ds.df.shape
  sum = 0
  #for index, row in df.iterrows():
  for _ in range(int(shape[0]*percentage)):
    #if random.uniform(0.,100.) < 5.:
    sum += shape[1] - ds.sizes[random.randint(0,shape[0]-1)]
  mean = sum/(shape[0]*percentage)
  for i in range(shape[1]-1,int(mean),-1):
    del df[i]
  for i in range(shape[0]):
    if ds.sizes[i] > mean:
        ds.sizes[i] = math.ceil(mean)
  return df

In [29]:
resizeDataSet(ds_gl,0.4)
ds_gl.df.shape

(20450, 103)

Comme vous pouvez le constater, nous avons réduit le nombre de colones du dataset afin de gagner du temps lors de l'exécution des algorithmes. 
Cette solution pose un problème dans le cas ou les datarecords sont ordonnées. En supprimant les colonnes supérieurs à la taille moyenne, nous pouvons perdre, par exemple, les valeurs les plus grandes dans le cadre d'un classement par ordre croissant.

## Troisième solution

Dans cette approche, nous réalisons une moyenne mobile pour l'attribution du poids. De cette manière, nous ne supprimons pas de valeurs dans le dataset. 

In [47]:
def algoFrequencesSolutionA(ds,nb_pattern)-> tuple:
  df = ds.df
  sizes = ds.sizes
  R = []
  IsInR = set()
  w = np.zeros(df.shape[0])
  totalW = 0
    
  # set les probas
  for i in range(len(w)):
    w[i] = math.pow(2,sizes[i])
    if i >= 2:
      w[i] = (w[i] + w[i-1] + w[i-2])/ 3
    totalW += w[i]
  # on selectionne 
  while len(R) < nb_pattern:
    random_row = random.uniform(0,totalW)
    # On cherche la ligne
    row = 0
    v = 0
    for i in range(len(w)):
      if  v > random_row:
        row = i - 1
        break
      v += w[i]
    # On selectionne un motif 
    pattern = np.array(df.iloc[row][:sizes[row]])
    random_v = random.randint(1, len(pattern) - 1 )
    for i in range(len(pattern)- random_v):
      pattern = np.delete(pattern, random.randint(0, len(pattern) - 1 ))
    # On ajoute seulement les motifs non présents dans l'ensemble R
    IsInR.add(np.array2string(pattern))
    if len(IsInR) != len(R):
      R.append(pattern)
  print(R)
  return R

In [52]:
dsetGL = pd.read_fwf("https://www.philippe-fournier-viger.com/spmf/datasets/LEVIATHAN.txt", sep=" ",header=None,)
df_gl= dsetGL[0].str.split(' ', expand=True)
ds_gl = DataSet(df_gl)
ds_gl.df.head()

,0,1,2,3,4,5,6,7,8,9,...,176,177,178,179,180,181,182,183,184,185
0,36,-1,37,-1,38,-1,17,-1,39,-1,...,None,None,None,None,None,None,None,None,None,None
1,36,-1,64,-1,17,-1,8,-1,65,-1,...,None,None,None,None,None,None,None,None,None,None
2,78,-1,79,-1,80,-1,81,-1,82,-1,...,None,None,None,None,None,None,None,None,None,None
3,155,-1,8,-1,156,-1,14,-1,157,-1,...,None,None,None,None,None,None,None,None,None,None
4,1,-1,1,-1,1,-1,172,-1,173,-1,...,None,None,None,None,None,None,None,None,None,None


In [53]:
patterns = algoFrequencesSolutionA(ds_gl,10)
fre = frequences(ds_gl,patterns)
for f in fre:
    print(f"frequence {round(f*100.0,4)}%")

[array(['206', '-1', '54', '-1', '-1'], dtype=object), array(['310', '-1', '20', '-1', '-1', '-1', '731', '-1', '18', '-1', '96',
       '-1', '4466', '5776', '-1', '8', '193', '-2'], dtype=object), array(['14', '-1', '436', '-1', '219', '-1', '355', '-1', '58', '567',
       '-1', '568', '41', '-1', '71', '-1', '569', '-1', '14', '-1',
       '570', '-1', '220', '-1', '571', '-1', '468', '-1', '572', '-1',
       '573', '-1', '18', '-1', '41', '-1', '14', '-1', '574', '-1',
       '575', '18', '-1', '-1', '319', '-1', '576', '-1', '577', '-1',
       '578', '579', '-1', '30', '-1', '290', '-1', '52', '-1', '45',
       '-1', '23', '-1', '41', '-1', '580', '-1', '29', '-1', '278', '18',
       '-1', '398', '-1', '219', '355', '21', '-1', '58', '441', '-1',
       '-2'], dtype=object), array(['-1', '1', '-1', '-1', '-1', '-1', '1', '-1', '1', '-1', '-1', '1',
       '-1', '1', '1', '2369', '-1', '-1', '-1', '-1', '-1', '1', '363',
       '360', '1921', '-1', '1', '-1', '-1', '1', '-1', 

En utilisant cette méthode, nous obtenons une fréquence proche entre les diverses échantillons. La diversité est présente, mais il semblerait que cette solution ne soit pas aussi efficace qu'esperé. 